<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Introduction

This notebook updates the descriptive dataframe for a subsequent modeling or evaluating task. This descriptive dataframe serves as the leading input to the modeling pipeline and contains all information that is necessary to create the training and evaluating datasets.

The steps within this workflow are as follows:

1. accordingly to the initialized path the function loads the descriptive dataframe
2. the function splits the instances per SNR, per machine and per ID into training and testing and creates an additional column
3. the column is being added to the descriptive dataframe and saved back to the location

To use this notebook you will have to do the following steps:

1. define the path to the descriptive dataframe (path='....')
2. run all the cells after that

In [5]:
import pandas as pd
BASE_FOLDER = '../../'
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load detection_pipe


In [6]:
import glob
diagram = 'extdia_v1'
path = BASE_FOLDER + '/dataset/{}'.format(diagram)

files = [f for f in glob.glob(path + "**/*pandaDisc*.pkl", recursive=True)]

In [7]:
files

['../..//dataset\\extdia_v1\\fan6dB00_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fan6dB02_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fan6dB04_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fan6dB06_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fanmin6dB00_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fanmin6dB02_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fanmin6dB04_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\fanmin6dB06_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pump6dB02_EDiaV1HPaug0_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pump6dB02_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pump6dB04_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pump6dB06_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl',
 '../..//dataset\\extdia_v1\\pumpmin6dB02_EDiaV1HP_pandaDisc.pkl',
 '../..//dataset\\

In [8]:
for file in files:
    tt_split(file)


../..//dataset\extdia_v1\fan6dB00_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fan6dB02_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fan6dB04_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fan6dB06_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fanmin6dB00_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fanmin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fanmin6dB04_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\fanmin6dB06_EDiaV1HP_pandaDisc.pkl --> Done
../..//dataset\extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done


ValueError: test_size=2 should be either positive and smaller than the number of samples 2 or a float in the (0, 1) range

In [11]:
with open('../..//dataset\\extdia_v1\\pump6dB02_EDiaV1HPaug0_outpICA_demix.pkl', 'rb') as f:
    df = pickle.load(f)

AttributeError: Can't get attribute 'feature_extractor_type' on <module '__main__' from '..\\..\\utility\\modeling\\JupyterLoad_modeling.py'>

In [10]:
df

NameError: name 'df' is not defined